In [10]:
import pywt
import os
import numpy as np
import pandas as pd

import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 

import matplotlib.pyplot as plt
plt.style.use('default')
plt.rcParams['axes.grid'] = True
plt.rcParams["figure.figsize"] = (16, 3)
plt.rcParams['figure.dpi'] = 150

import warnings
warnings.filterwarnings("ignore")

### Loading the UCI-HAR time-series dataset

In [2]:
def read_signals_ucihar(filename):
    with open(filename, 'r') as fp:
        data = fp.read().splitlines()
        data = map(lambda x: x.rstrip().lstrip().split(), data)
        data = [list(map(float, line)) for line in data]
    return data


def read_labels_ucihar(filename):
    with open(filename, 'r') as fp:
        activities = fp.read().splitlines()
        activities = list(map(int, activities))
    return activities


def load_ucihar_data(folder):
    train_folder = folder + 'train/Inertial Signals/'
    test_folder = folder + 'test/Inertial Signals/'
    labelfile_train = folder + 'train/y_train.txt'
    labelfile_test = folder + 'test/y_test.txt'
    train_signals, test_signals = [], []

    for input_file in os.listdir(train_folder):
        signal = read_signals_ucihar(train_folder + input_file)
        train_signals.append(signal)
    train_signals = np.transpose(np.array(train_signals), (1, 2, 0))

    for input_file in os.listdir(test_folder):
        signal = read_signals_ucihar(test_folder + input_file)
        test_signals.append(signal)
    test_signals = np.transpose(np.array(test_signals), (1, 2, 0))
    train_labels = read_labels_ucihar(labelfile_train)
    test_labels = read_labels_ucihar(labelfile_test)

    return train_signals, train_labels, test_signals, test_labels

folder_ucihar = 'UCI HAR Dataset/'
train_signals_ucihar, train_labels_ucihar, test_signals_ucihar, test_labels_ucihar = \
    load_ucihar_data(folder_ucihar)

### Applying the WT on the dataset and transforming the data to the right format
Bởi vì mỗi signal gồm 9 components khác nhau (coi như 9 sensors), câu hỏi là làm sao ta cho 9 ảnh này vào CNN

1. Không thể train separately vì 9 data này có liên quan đến nhau

2. Không thể nối vào hết với nhau vì đoạn nối sẽ là điểm coi như bị noise

3. Có thể kết hợp 9 ảnh thành 1 ảnh với 9 channels, cũng giống như ảnh thường có 3 channels, CNN có thể làm được điều này

<p align="center" width="100%">
    <img width="70%" src="https://ataspinar.com/wp-content/uploads/2018/12/9layer_image_CNN.png">
</p>

In [4]:
scales = range(1,128)
waveletname = 'morl'
train_size = 50
test_size= 5

train_data_cwt = np.ndarray(shape=(train_size, 127, 127, 9))  # creat a training matrix

for ii in range(0, train_size):
    for jj in range(0, 9):
        signal = train_signals_ucihar[ii, :, jj]
        coeff, freq = pywt.cwt(signal, scales, waveletname, 1)
        coeff_ = coeff[:,:127]
        train_data_cwt[ii, :, :, jj] = coeff_

test_data_cwt = np.ndarray(shape=(test_size, 127, 127, 9))
for ii in range(0,test_size):
    for jj in range(0,9):
        signal = test_signals_ucihar[ii, :, jj]
        coeff, freq = pywt.cwt(signal, scales, waveletname, 1)
        coeff_ = coeff[:,:127]
        test_data_cwt[ii, :, :, jj] = coeff_

uci_har_labels_train = list(map(lambda x: int(x) - 1, train_labels_ucihar))
uci_har_labels_test = list(map(lambda x: int(x) - 1, test_labels_ucihar))

x_train = train_data_cwt
y_train = list(uci_har_labels_train[:train_size])
x_test = test_data_cwt
y_test = list(uci_har_labels_test[:test_size])

In [12]:
num_classes = 7

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [14]:
x_train.shape, y_train.shape

((50, 127, 127, 9), (50, 7))

In [13]:
history = History()

img_x = 127
img_y = 127
img_z = 9
input_shape = (img_x, img_y, img_z)

batch_size = 16

epochs = 10

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])

train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))

Epoch 1/10


2022-10-20 23:11:25.254336: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


4/4 [==============================] - 1s 213ms/step - loss: 3.4072 - accuracy: 0.3200 - val_loss: 12.4355 - val_accuracy: 0.0000e+00
Epoch 2/10
4/4 [==============================] - 1s 159ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 47.5377 - val_accuracy: 0.0000e+00
Epoch 3/10
4/4 [==============================] - 1s 143ms/step - loss: 0.0126 - accuracy: 1.0000 - val_loss: 67.1559 - val_accuracy: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 1s 146ms/step - loss: 2.2912e-06 - accuracy: 1.0000 - val_loss: 82.5685 - val_accuracy: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 1s 145ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 94.1106 - val_accuracy: 0.0000e+00
Epoch 6/10
4/4 [==============================] - 1s 145ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 102.5172 - val_accuracy: 0.0000e+00
Epoch 7/10
4/4 [==============================] - 1s 150ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 108.5469 - val